I've initialized the data generating a tensor of 64 element of dimension 6, half of them will be symmetric and half of them not. I then created another tensor with the label (1 if symmetric 0 if it's not).

In [1]:
import torch
import numpy as np

from random import randrange
data = torch.zeros([64,6])

for i in range(64):
    for j in range(3):
        data[i,j] = randrange(10)

for i in range(1,64,2):
    for j in range(1,4):
        data[i,-j] = data[i,j-1]

for i in range(0,64,2):
    for j in range(3,6):
        data[i,j] = randrange(10)

target = torch.Tensor([i%2 for i in range(64)])
target = target.reshape(64,1)

data= torch.cat((data, target), dim=1)
data= data[torch.randperm(data.size()[0])]

X=data[0:50,0:6]
y=data[0:50,6]
y=y.reshape(50,1)
X_test=data[50:64,0:6]
y_test=data[50:64,6]
y_test=y_test.reshape(14,1)



The model is composed by two layer, for the activation function I opted for ReLU, while for the output layer I've used the sigmoid since we are dealing with a binary classification. 

In [2]:
class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(6,2, bias = True)
        self.layer2 = torch.nn.Linear(2,1, bias = True)

    def forward(self, X): 
        out = self.layer1(X)
        out = torch.nn.functional.relu(out)
        out = self.layer2(out)
        out = torch.nn.functional.relu(out)
        out = torch.sigmoid(out) 
        return out

I've implemented the training very similar as seen in class, I did not use the dataloader since I'm still not totally certain of how to initialize.

In [3]:
def train_epoch(model, X,y, loss_fn, optimizer):

     optimizer.zero_grad() 

     y_hat = model(X)


     loss = loss_fn(y_hat, y)

     loss.backward()

     optimizer.step()

     return model


In [4]:
def train_model(model, X,y, loss_fn, optimizer, num_epochs):

    model.train()
    for epoch in range(num_epochs):
        model = train_epoch(model, X,y, loss_fn, optimizer)
    return model


As stated in the paper the learning rate is set to 0.1, for the loss function I opted for the binary cross entropy which I thought was the most coherent with the problem, as it was requested I've used the gradient descendent for the optimizer. I've also initialized the weight as suggested from the paper

In [5]:
learn_rate = 0.1
num_epochs = 2000
model = MLP()
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate)

In [6]:
state_dict = model.state_dict()
state_dict["layer1.weight"] = (0.3 + 0.3) * torch.rand(2, 6) - 0.3
state_dict["layer2.weight"] = (0.3 + 0.3) * torch.rand(1, 2) - 0.3
model.load_state_dict(state_dict)


<All keys matched successfully>

In [7]:
 final_model = train_model(model, X,y, loss_fn, optimizer, num_epochs)

Here are the final weight, we can see that some of them are opposite (as it was in the paper), and also as in the paper the bias are all negative.

In [8]:
state_dict = final_model.state_dict()
state_dict

OrderedDict([('layer1.weight',
              tensor([[-0.7802,  1.9347, -1.0833,  0.9722, -1.8873,  0.8602],
                      [ 0.1014,  1.4374, -1.3245, -0.0651, -0.7436,  0.8644]])),
             ('layer1.bias', tensor([-0.3103,  0.2813])),
             ('layer2.weight', tensor([[-2.6809,  1.6138]])),
             ('layer2.bias', tensor([-0.0024]))])

In [9]:
def error_rate(model,X,y):
    tmp = model(X)
    y_hat = torch.Tensor([(0 if i.item() <= 0.5 else 1)for i in tmp])
    y_hat=y_hat.unsqueeze(-1)
    match_ground_truth = y_hat == y # -> tensor of booleans
    correct_matches = match_ground_truth.sum()
    return 1-(correct_matches / y_hat.shape[0]).item()

In [10]:
error_rate(final_model,X_test,y_test)

0.2857142686843872